In [23]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session
from sagemaker.inputs import TrainingInput

In [4]:
bucket_name = 'bankapplicationawsbucket' ## < --- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
# Retreive region name 
myRegion = boto3.session.Session().region_name
print(myRegion)

ap-south-1


In [8]:
## Create S3 bucket if not created already

## Access to S3 as a resource
s3 = boto3.resource('s3')

try:
    if myRegion=='ap-south-1':
        s3.create_bucket(Bucket=bucket_name,
                         CreateBucketConfiguration={'LocationConstraint': 'ap-south-1'}
                        )
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 Error', e)

S3 bucket created successfully


In [9]:
# set an output path where the trained xgboost model will be
prefix = 'xgboost-as-a-built-in-algo'
output_path = "s3://{}/{}/output".format(bucket_name, prefix)
print(output_path)

s3://bankapplicationawsbucket/xgboost-as-a-built-in-algo/output


### Downloading the dataset and storing in S3

In [14]:
import pandas as pd
import urllib

try:
    urllib.request.urlretrieve(url="https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", filename="bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error:',r)
    
try:
    model_data = pd.read_csv('./bank_clean.csv', index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data Load error:', e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [16]:
### Train Test split
import numpy as np

train_data, test_data = np.split(model_data.sample(frac=1, random_state=123), [int(0.7*len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [19]:
import os

os.path.join(prefix, 'train/train.csv')

'xgboost-as-a-built-in-algo/train/train.csv'

In [24]:
## Saving train and test data into buckets
# We start with train data
import os

## For running models from AWS Sagemaker, we need to put dependent variable as first column
pd.concat([train_data['y_yes'], train_data.drop(['y_yes', 'y_no'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [28]:
# Test data into buckets

## For running models from AWS Sagemaker, we need to put dependent variable as first column
pd.concat([test_data['y_yes'], test_data.drop(['y_yes', 'y_no'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

### Building XGboost model - In built algorithm

In [29]:
## Inbuilt algo present here are present as containers and images by sagemaker
## We need to pull that image in our instance (Instance means my local system or the instance currently running in sagemaker)

### This line automatically looks for the XGBoost image URI and builds an XGBoost container.
## specify the repo version depending on your preference

container = get_image_uri(boto3.Session().region_name,
                         repo_name='xgboost',
                         repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [39]:
# We can do hyperparameter tuning as well, 
#but it might get into issues with billing, 
# as hyperparamter tunnig takes a lot of time which will be added in our bill

# so here we are selecting the best parameter features, as per our data
## Need to be key value rules
hyperparameters = {
    "max_depth":"5",
    "eta":"0.2",
    "gamma":"4",
    "min_child_weight":"6",
    "subsample":"0.7",
    "objective":"binary:logistic",
    "num_round":50
}



In [41]:
# construct a Sagemaker estimator that calls the xgboost-container

estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1,
                                          instance_type='ml.m5.2xlarge',
                                          volume_size=5, # 5 GB
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait = 600
                                         )

In [42]:
estimator.fit({'train': s3_input_train, 'validation':s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-06-23-13-38-02-437


2024-06-23 13:38:02 Starting - Starting the training job...
2024-06-23 13:38:30 Starting - Preparing the instances for training...
2024-06-23 13:38:51 Downloading - Downloading input data...
2024-06-23 13:39:06 Downloading - Downloading the training image...
2024-06-23 13:40:02 Training - Training image download completed. Training in progress.
2024-06-23 13:40:02 Uploading - Uploading generated training model[2024-06-23 13:39:54.098 ip-10-0-182-58.ap-south-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is

### Deploy Machine Learning model

In [43]:
xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-06-23-13-44-48-795
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-06-23-13-44-48-795
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-06-23-13-44-48-795


-----!

### Prediction of the Test data

In [61]:
from sagemaker.serializers import CSVSerializer

test_data_array = test_data.drop(['y_no','y_yes'], axis=1).values  # load the data into array
xgb_predictor.content_type = 'text/csv'  # set data type for an inference
xgb_predictor.serializer = CSVSerializer() # set the serializer type

predictions = xgb_predictor.predict(test_data_array).decode("utf-8") # predict in string!
predictions_array = np.fromstring(predictions, sep=',') # and then turn these predictions into array
print(predictions_array.shape)

(12357,)


In [69]:
print(test_data.shape)
print(np.unique(np.round(predictions_array)))

(12357, 61)
[0. 1.]


In [67]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]
tp = cm.iloc[1,1]
fn = cm.iloc[1,0]
fp = cm.iloc[0,1]

acc = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", acc))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.8%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10824)    37% (162)
Purchase        9% (1099)     63% (272) 



### Deleting the endpoints

In [74]:
# Delete the endpoint as soon as its done, to have less billing
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()  # Eveyrthing in the bucket will get deleted

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2024-06-23-13-44-48-795


[{'ResponseMetadata': {'RequestId': 'RPVXDQYES5PDHNPW',
   'HostId': 'FQD5uTIVRh3X5PRKNSOk3y2X054OE482TYedBeZqtkISzkUvQCYDEiGiDjAuruf+6EQdZsxis+Y=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'FQD5uTIVRh3X5PRKNSOk3y2X054OE482TYedBeZqtkISzkUvQCYDEiGiDjAuruf+6EQdZsxis+Y=',
    'x-amz-request-id': 'RPVXDQYES5PDHNPW',
    'date': 'Sun, 23 Jun 2024 17:26:24 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-06-23-13-38-02-437/debug-output/index/000000000/000000000040_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-06-23-13-38-02-437/debug-output/index/000000000/000000000010_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-06-23-13-38-02-437/debug-output/claim.smd'},
   {'Key': 'xgboost-as-a-built-in-algo/test/test.cs